# Imports

In [1]:
from regression_model_comparison import RegressionModelComparison
import numpy as np
import pandas as pd
import mlflow
import time

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Data

In [3]:
dataX = pd.read_csv('data/engie_X.csv', header=0, sep=';', decimal='.')
# dataX.info()

In [4]:
dataY = pd.read_csv('data/engie_Y.csv', header=0,  sep=';', decimal='.')
dataY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617386 entries, 0 to 617385
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      617386 non-null  int64  
 1   TARGET  617386 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 9.4 MB


In [5]:
data_raw = pd.merge(dataX, dataY, on='ID', how='inner')
print("SHAPE = ", data_raw.shape)
data_raw.head(3)

SHAPE =  (617386, 79)


,ID,MAC_CODE,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,...,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std,Absolute_wind_direction_c,Nacelle_angle_c,TARGET
0,1,WT3,1.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,294.19000,294.23999,-0.703
1,2,WT3,2.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,297.82999,294.23999,-0.747
2,3,WT3,3.0,92.470001,92.470001,92.470001,0.0,7.0,7.0,7.0,...,0.0,0.0,0.0,2.4,2.4,2.4,0.0,322.20999,294.23999,-0.791


## Séparation des éoliennes

In [6]:
data_raw.groupby('MAC_CODE').agg(nrows=('MAC_CODE', 'count'))

,nrows
MAC_CODE,
WT1,154707
WT2,154791
WT3,154253
WT4,153635


## Observation des données

In [10]:
# if np.isnan(data_raw).any():
#     raise ValueError("Il y a des valeurs manquantes (NaN) dans les données")
# else:
#     print("Aucune valeur manquante (NaN) dans les données")

In [ ]:
import statsmodels.api as sm

#Fit linear model to any dataset
model = sm.OLS(Y, X)
results = model.fit()

#create instance of influence
influence = results.get_influence()

#leverage (hat values)
leverage = influence.hat_matrix_diag

# Modelisation

Objectif = Un modèle gagnant pour chaque éolienne  

Il serait judicieux de séparer les JDD de chaque éolienne en groupes via clustering (car les JDD sont encore gros : 150k) pour trouver des groupes au sein des JDD de chaque éolienne.  

En attendant un modèle gagnant par éolienne, puis on prédit sur toutes les lignes de chaque éolienne, à la fin on a nos 350k prévisions et on calcule la MAE finale

## Echantillon d'essais et de debuguage

In [9]:
sample_size = 50
X = data_raw.drop(columns=['ID', 'MAC_CODE', 'TARGET']).iloc[:sample_size, :]
Y = data_raw.TARGET.iloc[:sample_size]

In [10]:
comparison = RegressionModelComparison(
    X,
    Y,
    scorings=['mae', 'mse'],
    test_size=0.1,
    seed=3,
    mlflow=False
    )

##### Splitting Dataset with test_size = 0.1 and random_state = 3


In [11]:
numerical_features = [
    'Date_time', 'Pitch_angle_std', 'Hub_temperature',
       'Hub_temperature_min', 'Hub_temperature_max', 'Hub_temperature_std',
       'Generator_converter_speed', 'Generator_converter_speed_min',
       'Generator_converter_speed_max', 'Generator_converter_speed_std',
       'Generator_speed', 'Generator_speed_min', 'Generator_speed_max',
       'Generator_speed_std', 'Generator_bearing_1_temperature',
       'Generator_bearing_1_temperature_min',
       'Generator_bearing_1_temperature_max',
       'Generator_bearing_1_temperature_std',
       'Generator_bearing_2_temperature',
       'Generator_bearing_2_temperature_min',
       'Generator_bearing_2_temperature_max',
       'Generator_bearing_2_temperature_std', 'Generator_stator_temperature',
       'Generator_stator_temperature_min', 'Generator_stator_temperature_max',
       'Generator_stator_temperature_std', 'Gearbox_bearing_1_temperature',
       'Gearbox_bearing_1_temperature_min',
       'Gearbox_bearing_1_temperature_max',
       'Gearbox_bearing_1_temperature_std', 'Gearbox_bearing_2_temperature',
       'Gearbox_bearing_2_temperature_min',
       'Gearbox_bearing_2_temperature_max',
       'Gearbox_bearing_2_temperature_std', 'Gearbox_inlet_temperature',
       'Gearbox_inlet_temperature_min', 'Gearbox_inlet_temperature_max',
       'Gearbox_inlet_temperature_std', 'Gearbox_oil_sump_temperature',
       'Gearbox_oil_sump_temperature_min', 'Gearbox_oil_sump_temperature_max',
       'Gearbox_oil_sump_temperature_std', 'Nacelle_angle',
       'Nacelle_angle_min', 'Nacelle_angle_max', 'Nacelle_angle_std',
       'Nacelle_temperature', 'Nacelle_temperature_min',
       'Nacelle_temperature_max', 'Nacelle_temperature_std',
       'Absolute_wind_direction', 'Outdoor_temperature',
       'Outdoor_temperature_min', 'Outdoor_temperature_max',
       'Outdoor_temperature_std', 'Grid_frequency', 'Grid_frequency_min',
       'Grid_frequency_max', 'Grid_frequency_std', 'Grid_voltage',
       'Grid_voltage_min', 'Grid_voltage_max', 'Grid_voltage_std',
       'Rotor_speed', 'Rotor_speed_min', 'Rotor_speed_max', 'Rotor_speed_std',
       'Rotor_bearing_temperature', 'Rotor_bearing_temperature_min',
       'Rotor_bearing_temperature_max', 'Rotor_bearing_temperature_std'
       ]

other_features = ['Pitch_angle', 'Pitch_angle_min', 'Pitch_angle_max', 'Absolute_wind_direction_c', 'Nacelle_angle_c']

In [12]:
comparison.preprocessing(
    numerical_features,
    other_features,
    nknots=4,
    poly_order=2,
    scaler='standard'
    )

In [13]:
df_results = comparison.run_comparison(
                    preproc=['base'],
                    model_param={
                        'linear_regression': {},
                        # 'ridge': {},
                        'lasso': {},
                        'elasticnet': {},
                        'randomforest': {
                            'regressor__n_estimators' : [100, 1000], # Nombre d'arbres dans la forêt. defaut 100
                            'regressor__max_depth' : [None, 30], # Profondeur maximale des arbres. Si None, les arbres sont développés jusqu'à ce que toutes les feuilles soient pures ou que chaque feuille contienne moins que min_samples_split échantillons
                            'regressor__max_features': [3, 'sqrt'], # Nombre maximum de caractéristiques considérées pour chaque split (division d'un nœud en deux sous-nœuds)
                            },
                        # 'grd_boosting': {
                        #     'regressor__learning_rate' : [.01, 1],
                        #     'regressor__max_depth' : [3, 9],
                        #     'regressor__subsample' : [0.5, 1],
                        #     'regressor__n_estimators' : [100, 1000]
                        #     }
                        },
                    nfolds=5,
                    verbose=False
                    )

###### Start comparison ######
Using preprocessor : base
Using regressor : linear_regression


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/sklearn/pipeline.py", line 386, in _fit
    self._validate_steps()
  File "/opt

In [32]:
df_results[df_results.mae_test == df_results.mae_test.min()]

,model_name,mae_test,mae_train,mse_test,mse_train,model,params
3,randomforest,0.652278,0.196032,1.265389,0.181485,"(ColumnTransformer(transformers=[('num',\n ...","{'regressor__max_depth': None, 'regressor__max..."


### Results

In [ ]:
pd.DataFrame(data=[best_estimator_list, predict_score_list, best_score_list, best_params_list], index=['model', 'test_score', 'train_val_score', 'params']).T

## 1ère éolienne

In [ ]:
data_wt1 = data_raw[data_raw.MAC_CODE == 'WT1']
data_wt1.shape

(154707, 79)

### Sur 5k données

In [11]:
sample_size = 5000
X = data_wt1.drop(columns=['ID', 'MAC_CODE']).iloc[:sample_size, :]
Y = data_wt1.TARGET.iloc[:sample_size]

In [ ]:
comparison = RegressionModelComparison(
    X,
    Y,
    scorings=['mae', 'mse'],
    test_size=0.1,
    seed=3,
    mlflow=True
    )

comparison.preprocessing(
    numerical_features,
    other_features,
    nknots=4,
    poly_order=2,
    scaler='standard',
    verbose=False
    )

##### Splitting Dataset with test_size = 0.1 and random_state = 3


In [16]:
best_params_wt1, best_score_wt1, best_estimator_wt1, predict_score_wt1 = comparison.run_comparison(
        preproc=['base', 'poly', 'splines'],
        model_param={
            'linear_regression': {},
            # 'ridge': {},
            'lasso': {},
            'elasticnet': {},
            'randomforest': {
                'regressor__n_estimators' : [100, 300], # Nombre d'arbres dans la forêt. defaut 100
                'regressor__max_depth' : [None, 20], # Profondeur maximale des arbres. Si None, les arbres sont développés jusqu'à ce que toutes les feuilles soient pures ou que chaque feuille contienne moins que min_samples_split échantillons
                'regressor__max_features': [3, 'sqrt'], # Nombre maximum de caractéristiques considérées pour chaque split (division d'un nœud en deux sous-nœuds)
                },
            'grd_boosting': {
                'regressor__learning_rate' : [.01, 1],
                'regressor__max_depth' : [3, 7],
                'regressor__subsample' : [0.5, 1],
                'regressor__n_estimators' : [100, 300]
                }
            },
        nfolds=5,
        verbose=False
        )

###### Start comparison ######
Using preprocessor : ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Date_time', 'Pitch_angle', 'Pitch_angle_min',
                                  'Pitch_angle_max', 'Pitch_angle_std',
                                  'Hub_temperature', 'Hub_temperature_min',
                                  'Hub_temperature_max', 'Hub_temperature_std',
                                  'Generator_converter_speed',
                                  'Generator_converter_speed_min',
                                  'Generator_c...
                                  'Generator_bearing_1_temperature_std',
                                  'Generator_bearing_2_temperature',
                                  'Generator_bearing_2_temperature_min',
                                  'Generator

#### Results

In [23]:
data_wt1[['TARGET']].describe()

,TARGET
count,154707.000000
mean,385.726388
std,471.248086
min,-19.469001
25%,29.772499
50%,208.550000
75%,566.952011
max,2250.997000


In [17]:
df_results_wt1 = pd.DataFrame(data=[best_estimator_wt1, predict_score_wt1, best_score_wt1, best_params_wt1], index=['model', 'test_score', 'train_val_score', 'params']).T
df_results_wt1

,model,test_score,train_val_score,params
0,"(ColumnTransformer(transformers=[('num',\n ...",-86.871302,-84.620638,{}
1,LassoCV(),0.466873,0.47153,"{'alphas': None, 'copy_X': True, 'cv': None, '..."
2,ElasticNetCV(),0.878193,0.919668,"{'alphas': None, 'copy_X': True, 'cv': None, '..."
3,"(ColumnTransformer(transformers=[('num',\n ...",-14.030577,-16.223314,"{'regressor__max_depth': None, 'regressor__max..."
4,"(ColumnTransformer(transformers=[('num',\n ...",-14.493956,-16.579571,"{'regressor__learning_rate': 1, 'regressor__ma..."
5,"(ColumnTransformer(transformers=[('num',\n ...",-87.070081,-84.773728,{}
6,LassoCV(),0.466873,0.47153,"{'alphas': None, 'copy_X': True, 'cv': None, '..."
7,ElasticNetCV(),0.878193,0.919668,"{'alphas': None, 'copy_X': True, 'cv': None, '..."
8,"(ColumnTransformer(transformers=[('num',\n ...",-14.745343,-16.78359,"{'regressor__max_depth': None, 'regressor__max..."
9,"(ColumnTransformer(transformers=[('num',\n ...",-13.818128,-16.668648,"{'regressor__learning_rate': 1, 'regressor__ma..."


In [25]:
df_results_wt1.iloc[9, :].params

{'regressor__learning_rate': 1,
 'regressor__max_depth': 7,
 'regressor__n_estimators': 300,
 'regressor__subsample': 1}

In [30]:
df_results_wt1.iloc[9, :]

model              (ColumnTransformer(transformers=[('num',\n    ...
test_score                                                -13.818128
train_val_score                                           -16.668648
params             {'regressor__learning_rate': 1, 'regressor__ma...
Name: 9, dtype: object

### Sur toutes les données : on prend le modèle gagnant uniquement !

Est-ce qu'on le rethune ?

In [33]:
X = data_wt1.drop(columns=['ID', 'MAC_CODE'])
Y = data_wt1.TARGET

X.shape

(154707, 77)

In [31]:
model_wt1 = df_results_wt1.iloc[9, :].model
model_wt1

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Date_time', 'Pitch_angle',
                                                   'Pitch_angle_min',
                                                   'Pitch_angle_max',
                                                   'Pitch_angle_std',
                                                   'Hub_temperature',
                                                   'Hub_temperature_min',
                                                   'Hub_temperature_max',
                                                   'Hub_temperature_std',
                                                   'Generator_converter_speed',
                                                   'Generator_conver...
                                                   'Generator_stator_temperature_max',
                                                   'Generator_stator_temperature_std',
                                                   'Gearbox_bearing_1_temperature', ...]),
                                                 ('poly',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('poly',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Absolute_wind_direction_c',
                                                   'Nacelle_angle_c'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=1, max_depth=7,
                                           n_estimators=300, subsample=1))])

In [37]:
X_train_wt1, X_test_wt1, y_train_wt1, y_test_wt1 = train_test_split(X, Y, test_size=0.2, random_state=3)

print(y_train_wt1.head(3))
X_train_wt1.head(3)

564692     29.426
532387     48.203
493592    107.900
Name: TARGET, dtype: float64


,Date_time,Pitch_angle,Pitch_angle_min,Pitch_angle_max,Pitch_angle_std,Hub_temperature,Hub_temperature_min,Hub_temperature_max,Hub_temperature_std,Generator_converter_speed,...,Rotor_speed_min,Rotor_speed_max,Rotor_speed_std,Rotor_bearing_temperature,Rotor_bearing_temperature_min,Rotor_bearing_temperature_max,Rotor_bearing_temperature_std,Absolute_wind_direction_c,Nacelle_angle_c,TARGET
564692,104713.0,-1.00,-1.00,-1.00,0.0,11.0,11.0,11.0,0.0,926.50000,...,8.770001,8.82,0.01,20.620001,20.200001,20.600000,0.13,187.289990,194.250000,29.426
532387,72313.0,-1.01,-1.01,-1.01,0.0,20.0,20.0,20.0,0.0,926.84003,...,8.770001,8.93,0.03,30.590000,30.500000,30.700001,0.08,29.559999,20.830000,48.203
493592,32396.0,-1.00,-1.00,-1.00,0.0,29.0,29.0,29.0,0.0,1027.64000,...,8.760000,10.46,0.42,36.500000,36.500000,36.599998,0.00,27.870001,29.610001,107.900


In [ ]:
model_wt1.fit(X_train_wt1, y_train_wt1)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Date_time', 'Pitch_angle',
                                                   'Pitch_angle_min',
                                                   'Pitch_angle_max',
                                                   'Pitch_angle_std',
                                                   'Hub_temperature',
                                                   'Hub_temperature_min',
                                                   'Hub_temperature_max',
                                                   'Hub_temperature_std',
                                                   'Generator_converter_speed',
                                                   'Generator_conver...
                                                   'Generator_stator_temperature_max',
                                                   'Generator_stator_temperature_std',
                                                   'Gearbox_bearing_1_temperature', ...]),
                                                 ('poly',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('poly',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Absolute_wind_direction_c',
                                                   'Nacelle_angle_c'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=1, max_depth=7,
                                           n_estimators=300, subsample=1))])

In [ ]:
prevision = model_wt1.predict(X_test_wt1)

In [ ]:
prevision_score = mean_absolute_error(y_true=y_test_wt1, y_pred=prevision)

print(prevision_score)

22.181820383933353


In [ ]:
estimation = model_wt1.predict(X_train_wt1)

mean_absolute_error(y_true=y_train_wt1, y_pred=estimation)

np.float64(3.6977470974365847)

In [44]:
mean_squared_error(y_true=y_test_wt1, y_pred=prevision)

np.float64(3497.215362884311)

Il faut :  
- Scripter la comparaison de modèle pour chaque éolienne sur 5k données (test_size = 0.1, random choice des 5k)
- Extraire le modèle gagnant
- Entraîner le modèle gagnant sur toutes les données de l'éolienne
- Prédire et créer un y_pred par éolienne
- Compiler les y_pred sur les 4 jeux de test
- Faire la MAE sur le jeu de test global

PREPARER LES DIFFERENTS JEUX EN AMONT  

AJOUTER LA MSE POUR INFO ?

# Comparaison complète

In [8]:
start_time = time.time()

sample_size = 5000
seed = 3
test_size = 0.1
metrics = ['mae', 'mse']

results_wt = {}
y_pred_wt = {}

array_wt = data_raw.MAC_CODE.unique()
array_wt.sort()

for eolienne in array_wt:
    print(f"### Comparison for wind turbine {eolienne}")

    results_wt[eolienne] = {}

    data_wt = data_raw[data_raw.MAC_CODE == eolienne]
    print(f"Shape of the dataframe = {data_wt.shape}")

    df_sample = data_wt.drop(columns=['ID', 'MAC_CODE']).sample(sample_size, random_state=seed)
    X = df_sample.drop(columns=['TARGET'])
    Y = df_sample.TARGET

    comparison = RegressionModelComparison(
        X,
        Y,
        scorings=metrics,
        test_size=test_size,
        seed=seed,
        mlflow=True
        )

    comparison.preprocessing(
        numerical_features,
        other_features,
        nknots=4,
        poly_order=2,
        scaler='standard',
        verbose=False
        )
    
    df_results = comparison.run_comparison(
        preproc=['base', 'poly'],
        model_param={
            'linear_regression': {},
            # 'ridge': {},
            'lasso': {},
            'elasticnet': {},
            'randomforest': {
                'regressor__n_estimators' : [100, 300], # Nombre d'arbres dans la forêt. defaut 100
                'regressor__max_depth' : [None, 20], # Profondeur maximale des arbres. Si None, les arbres sont développés jusqu'à ce que toutes les feuilles soient pures ou que chaque feuille contienne moins que min_samples_split échantillons
                'regressor__max_features': [3, 'sqrt'], # Nombre maximum de caractéristiques considérées pour chaque split (division d'un nœud en deux sous-nœuds)
                },
            'grd_boosting': {
                'regressor__learning_rate' : [.01, 1],
                'regressor__max_depth' : [3, 7],
                'regressor__subsample' : [0.5, 1],
                'regressor__n_estimators' : [100, 300]
                }
            },
        nfolds=5,
        verbose=False
        )
    
    # Selection du meilleur modèle selon la MAE sur le jeu de test lors de la recherche sur grille
    best = df_results[df_results.mae_test == df_results.mae_test.min()]
    best_model = best.model.iloc[0]

    results_wt[eolienne]['best_model'] = best_model
    results_wt[eolienne]['best_params'] = best.params.iloc[0]

    # Préparation de l'ensemble du dataset de l'éolienne
    X_full = data_wt.drop(columns=['ID', 'MAC_CODE', 'TARGET'])
    Y_full = data_wt.TARGET

    # Train/test split sur l'ensemble du dataset de l'éolienne
    X_train_wt, X_test_wt, y_train_wt, y_test_wt = train_test_split(X_full, Y_full, test_size=0.3, random_state=seed)

    # Entraînement et test sur l'ensemble du dataset
    best_model.fit(X_train_wt, y_train_wt)
    estimation = best_model.predict(X_train_wt)
    prevision = best_model.predict(X_test_wt)

    # Sauvegarde du y_pred de l'éolienne
    y_pred_wt[eolienne] = best_model.predict(X_full)

    scores = {}
    for metric in metrics: 
        scores[metric] = {}
        if metric == 'mae':
            scores[metric]['train'] = mean_absolute_error(y_true=y_train_wt, y_pred=estimation)
            scores[metric]['test'] = mean_absolute_error(y_true=y_test_wt, y_pred=prevision)
            
        elif metric == 'mse':
            scores[metric]['train'] = mean_squared_error(y_true=y_train_wt, y_pred=estimation)
            scores[metric]['test'] = mean_squared_error(y_true=y_test_wt, y_pred=prevision)
    
    results_wt[eolienne]['metrics'] = scores

duration = time.time() - start_time
print(f"Total duration of computation = {duration / 60} minutes")

### Comparison for wind turbine WT1
Shape of the dataframe = (154707, 79)
##### Splitting Dataset with test_size = 0.1 and random_state = 3
###### Start comparison ######
Using preprocessor : base
Using regressor : linear_regression


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/sklearn/pipeline.py", line 386, in _fit
    self._validate_steps()
  File "/opt

In [ ]:
data_results = []
index_results = ['wt', 'model', 'mae_train', 'mae_test', 'mse_train', 'mse_test']

for wt in results_wt.keys():
    data_results_wt = [wt]
    best_model = str(type(results_wt['WT1']['best_model'].named_steps['regressor'])).split('.')[-1]
    data_results_wt.append(best_model)

    for metric in results_wt[wt]['metrics'].keys():
        for key, value in results_wt[wt]['metrics'][metric].items():
            data_results_wt.append(value)

    data_results.append(data_results_wt)

df_results_wt = pd.DataFrame(data_results, columns=index_results)
df_results_wt

,wt,model,mae_train,mae_test,mse_train,mse_test
0,WT1,RandomForestRegressor'>,5.355959,14.448514,168.988133,1263.563226
1,WT2,RandomForestRegressor'>,5.222051,12.225832,130.567500,834.449953
2,WT3,RandomForestRegressor'>,5.466036,14.623549,172.656806,1300.750919
3,WT4,RandomForestRegressor'>,4.597733,12.328863,119.961333,880.264643


In [2]:
y_pred_wt

NameError: name 'y_pred_wt' is not defined

REVOIR LA CONSTRUCTION DU Y FINAL !!!

In [26]:
y_pred_final = np.concat([y_pred_wt['WT1'], y_pred_wt['WT2'], y_pred_wt['WT3'], y_pred_wt['WT4']])
print(y_pred_final.shape)

(120,)


In [ ]:
mae_final = mean_absolute_error(y_true=data_raw.TARGET, y_pred=y_pred_final)
mse_final = mean_absolute_error(y_true=data_raw.TARGET, y_pred=y_pred_final)

{'WT3': array([-1.29920002, -1.44572001, -0.23748   , -1.35255   , -1.19888002,
        -1.26433   , -1.21769001, -0.71037   , -1.2926    , -2.13960001,
        -1.25068999, -0.10977   , -1.25222999, -1.29535003, -1.20207   ,
        -1.20415999, -1.25739999, -1.50853001, -1.4334    , -0.65119   ,
        -1.23341999, -4.11640998, -0.14596   , -0.65988   , -1.26850999,
        -1.38037998, -1.74789   , -1.41327   , -3.32771   , -2.97592999]),
 'WT2': array([-5.29721201, -4.56823295, -0.65263503, -6.04001029, -4.60083346,
        -5.05880515, -4.98272803, -3.5823149 , -5.36816668, -6.76014596,
        -4.56225663, -0.45431596, -4.74738186, -4.36967012, -4.78984963,
        -4.65051041, -4.11374107, -5.22507315, -6.28459497, -0.43718576,
        -4.61544655, -7.44156413, -0.39117555, -3.02585862, -5.45210994,
        -4.35234236, -6.33843262, -5.21134003, -5.97455589, -7.32296709]),
 'WT4': array([-0.14995118, -0.23182579,  0.08245548, -0.32972458, -0.46313291,
         0.03556702, -0.19